In [1]:
from sgkit.distance.api import pairwise_distance
import dask.array as da

from dask.distributed import Client, performance_report, get_task_stream
from dask.diagnostics import ProgressBar, ResourceProfiler
from bokeh.io import output_notebook

import fsspec
import zarr
import coiled

In [2]:
coiled.create_software_environment(
    name="sgkit-no-parallel",
    conda="environment.yml",
)

Updating software environment...
Found built software environment


In [3]:
coiled.create_cluster_configuration(name="sgkit-no-parallel", software='sgkit-no-parallel', worker_cpu=4, worker_memory="16 GiB")

In [5]:
cluster = coiled.Cluster(configuration="aktech/sgkit-no-parallel", n_workers=50, worker_options={
    "nthreads": 4,
    "env": {
#         "OMP_NUM_THREADS": "2",
#         "OPENBLAS_NUM_THREADS": "1",
    },
    "memory_limit": "16 GB"
})

client = Client(cluster)
print('Dashboard:', client.dashboard_link)

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found
Dashboard: http://ec2-18-223-172-75.us-east-2.compute.amazonaws.com:8787


In [7]:
store = fsspec.get_mapper('gs://ag1000g-release/phase2.AR1/variation/main/zarr/all/ag1000g.phase2.ar1')
callset_snps = zarr.open_consolidated(store=store)
gt = callset_snps['2R/calldata/GT']

In [8]:
gt_da = da.from_zarr(gt)
x = gt_da[:, :, 1].T
x = x.rechunk((100, 1000000))
x

dask.array<rechunk-merge, shape=(1142, 24767689), dtype=int8, chunksize=(100, 1000000), chunktype=numpy.ndarray>

In [10]:
da.ones

functools.partial(<function wrap_func_shape_as_first_arg at 0x7fae336a3310>, <function _broadcast_trick_inner at 0x7fae336a3550>, dtype='f8')

In [ ]:
# simulate some genotype data
import numpy as np
x = da.random.choice(np.array([0, 1, 2], dtype='i1'), 
                     p=[.7, .2, .1,], 
                     size=(1_000, 28_500_000), chunks=(1000, 100000))
x

In [9]:
%%time
with performance_report(filename="dask-report-16G-4cpu-4threads-random.html"), get_task_stream(plot='save', filename="task-stream-16G-4cpu-4threads-random.html") as ts:
    out = pairwise_distance(x)

CPU times: user 28.4 s, sys: 412 ms, total: 28.8 s
Wall time: 5min 59s


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
